# Test NHL Player Markets on The Odds API

Goal: Discover what player prop markets are actually available for NHL

In [2]:
import requests
import json
from pprint import pprint
import os

In [3]:
# SET YOUR API KEY HERE
API_KEY = "b81efa2d2e27c141b788a7fb02065fe0"  # Replace with your actual key

BASE_URL = "https://api.the-odds-api.com/v4"

## Step 1: Get today's NHL games

In [4]:
# Get upcoming NHL games
response = requests.get(
    f"{BASE_URL}/sports/icehockey_nhl/odds",
    params={
        "apiKey": API_KEY,
        "regions": "us",
        "markets": "h2h"
    }
)

print(f"Status: {response.status_code}")
games = response.json()
print(f"\nFound {len(games)} NHL games\n")

# Show first game
if games:
    game = games[0]
    print(f"First game:")
    print(f"  ID: {game['id']}")
    print(f"  {game['away_team']} @ {game['home_team']}")
    print(f"  Start: {game['commence_time']}")
    
    event_id = game['id']
else:
    print("No games found. NHL might be off-season or no games today.")
    event_id = None

Status: 200

Found 13 NHL games

First game:
  ID: c6b2ee3a1c69b08464c9b8262897874b
  Washington Capitals @ Detroit Red Wings
  Start: 2025-12-21T18:15:55Z


## Step 2: Check what markets are available for this event

In [5]:
if event_id:
    # Get available markets for this event
    markets_response = requests.get(
        f"{BASE_URL}/sports/icehockey_nhl/events/{event_id}/markets",
        params={"apiKey": API_KEY}
    )
    
    print(f"Status: {markets_response.status_code}")
    available_markets = markets_response.json()
    
    print(f"\nAvailable markets for this game:")
    print("=" * 50)
    for market in available_markets:
        print(f"  - {market}")
else:
    print("No event ID to check markets")

Status: 422

Available markets for this game:
  - message
  - error_code
  - details_url


## Step 3: Try to get player_points market

In [6]:
if event_id:
    # Try player_points
    player_points_response = requests.get(
        f"{BASE_URL}/sports/icehockey_nhl/events/{event_id}/odds",
        params={
            "apiKey": API_KEY,
            "regions": "us",
            "markets": "player_points"
        }
    )
    
    print(f"Status: {player_points_response.status_code}")
    
    if player_points_response.status_code == 200:
        data = player_points_response.json()
        print("\n✅ SUCCESS! player_points market EXISTS\n")
        
        # Show sample data
        if data.get('bookmakers'):
            bookmaker = data['bookmakers'][0]
            print(f"Sample from {bookmaker['key']}:")
            
            for market in bookmaker.get('markets', []):
                if market['key'] == 'player_points':
                    print(f"\nMarket: {market['key']}")
                    print("Sample players:")
                    
                    # Show first 3 players
                    outcomes = market.get('outcomes', [])
                    for i in range(0, min(6, len(outcomes)), 2):  # Over/Under pairs
                        if i < len(outcomes):
                            over = outcomes[i]
                            print(f"  {over.get('description', 'N/A')}: {over.get('point', 'N/A')} (O: {over.get('price', 'N/A')}, U: {outcomes[i+1].get('price', 'N/A') if i+1 < len(outcomes) else 'N/A'})")
    else:
        print("\n❌ player_points market NOT available")
        print(f"Error: {player_points_response.text}")
else:
    print("No event ID to test")

Status: 200

✅ SUCCESS! player_points market EXISTS



## Step 4: Try all known player markets

In [7]:
if event_id:
    # Known NHL player markets from docs
    test_markets = [
        "player_points",
        "player_power_play_points",
        "player_assists",
        "player_blocked_shots",
        "player_goal_scorer_anytime",
        "player_goal_scorer_first",
        "player_goal_scorer_last",
        "player_total_saves",
        "player_shots_on_goal"
    ]
    
    print("Testing all known NHL player markets...\n")
    print("=" * 60)
    
    working_markets = []
    
    for market in test_markets:
        test_response = requests.get(
            f"{BASE_URL}/sports/icehockey_nhl/events/{event_id}/odds",
            params={
                "apiKey": API_KEY,
                "regions": "us",
                "markets": market
            }
        )
        
        if test_response.status_code == 200:
            data = test_response.json()
            has_data = bool(data.get('bookmakers'))
            status = "✅ WORKS" if has_data else "⚠️  No data"
            print(f"{status:15} {market}")
            if has_data:
                working_markets.append(market)
        else:
            print(f"❌ INVALID      {market}")
    
    print("\n" + "=" * 60)
    print(f"\nWorking markets: {len(working_markets)}")
    print(f"Markets: {working_markets}")
else:
    print("No event ID to test")

Testing all known NHL player markets...

⚠️  No data     player_points
⚠️  No data     player_power_play_points
⚠️  No data     player_assists
⚠️  No data     player_blocked_shots
✅ WORKS         player_goal_scorer_anytime
✅ WORKS         player_goal_scorer_first
⚠️  No data     player_goal_scorer_last
⚠️  No data     player_total_saves
✅ WORKS         player_shots_on_goal


Working markets: 3
Markets: ['player_goal_scorer_anytime', 'player_goal_scorer_first', 'player_shots_on_goal']


## Step 5: Get full data for working market

In [10]:
# If player_points works, get full data
if event_id and 'player_points' in working_markets:
    full_response = requests.get(
        f"{BASE_URL}/sports/icehockey_nhl/events/{event_id}/odds",
        params={
            "apiKey": API_KEY,
            "regions": "us",
            "markets": "player_points"
        }
    )
    
    full_data = full_response.json()
    
    print("Full player_points data structure:\n")
    pprint(full_data, depth=4)

## Summary

Run all cells above to:
1. Check if NHL games exist today
2. Discover available markets
3. Test if player_points exists
4. Test all known player markets
5. Get sample data structure